In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/SIDwithIS/casestudies -b casestudies

# # To update the work environment
# %cd /content/drive/My Drive/SIDwithIS
# !git pull

%cd /content/drive/My Drive/SIDwithIS/casestudies

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../sid/")

from builder import Builder
from loader import Loader
from mylogger import MyLogger
from sid_agent import SidAgent
from sid_agent_factory import SidAgentFactory
from sid_build_parameter import SidBuildParameter
from sid_trainer_factory import SidTrainerFactory
from store import Store
from sid_evaluator import SidEvaluator
from builtins import isinstance
from cs01_environment_factory import Cs01EnvironmentFactory
from cs01_build_parameter_factory import Cs01BuildParameterFactory
from cs01_build_parameter import Cs01BuildParameter

In [ ]:
trainingHistoryDbPath = "cs01_training_history.sqlite"

In [ ]:
target_casestudy = "cs01a-01"

## S400: Load trained agents to analyze them

### SS410: initialize loader

In [ ]:
agentFactory = SidAgentFactory()
environmentFactory = Cs01EnvironmentFactory()
trainerFactory = SidTrainerFactory()

store = Store(trainingHistoryDbPath)

buildParameterFactory = Cs01BuildParameterFactory()
loader = Loader(agentFactory=agentFactory
                , environmentFactory=environmentFactory
                , trainerFactory=trainerFactory
                , buildParameterFactory=buildParameterFactory
                , store = store)

### SS420: initialize evaluator

In [ ]:
evaluator = SidEvaluator()

### SS420: check the distribution of sids

In [ ]:
tbl = {
    "epoch": []
    , "score": []
    , "criteria": []
}
for agent, buildParameter, epoch, environment, trainer in loader.load(target_casestudy + "%"):
    row = evaluator.evaluate(agent, buildParameter, epoch, environment, trainer)
    for criteria in row:
        tbl["epoch"].append(epoch)
        for key in buildParameter.__dict__:
            if not key in tbl:
                tbl[key] = []
            tbl[key].append(buildParameter.__dict__[key])
        tbl["criteria"].append(criteria)
        tbl["score"].append(row[criteria])
tbl = pd.DataFrame(tbl)

In [ ]:
tbl.to_csv("%s_score.csv" % target_casestudy)